In [1]:
import pandas as pd
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score,
    recall_score, classification_report,
    confusion_matrix
)
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# ======================================================
# 1. LOAD + BALANCE DATA
# ======================================================
df = pd.read_csv('/kaggle/input/memo-health-trigger-dataset/Memo_Dataset.csv')
df = df[['Question', 'Trigger']]
df['Trigger'] = df['Trigger'].astype(int)

min_count = min(5000, df['Trigger'].value_counts().min())
df_balanced = df.groupby('Trigger', group_keys=False).apply(
    lambda x: x.sample(min_count, random_state=42)
).reset_index(drop=True)

df_balanced = df_balanced.rename(columns={'Question': 'text', 'Trigger': 'label'})

train_df, test_df = train_test_split(
    df_balanced,
    test_size=0.2,
    stratify=df_balanced['label'],
    random_state=42
)

print("Train:", len(train_df), " Test:", len(test_df))

# ======================================================
# 2. LOAD MARBERT
# ======================================================
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "UBC-NLP/MARBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

# Freeze MARBERT
for param in model.parameters():
    param.requires_grad = False

# ======================================================
# 3. FUNCTION TO EXTRACT CLS EMBEDDINGS
# ======================================================
def get_cls_embeddings(texts):
    embeddings = []
    batch_size = 32

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]

        encoded = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoded)
            cls_vec = outputs.last_hidden_state[:, 0, :].cpu().numpy()

        embeddings.append(cls_vec)

    return np.vstack(embeddings)

# ======================================================
# 4. EMBEDDINGS FOR TRAIN + TEST
# ======================================================
print("Extracting TRAIN embeddings...")
X_train = get_cls_embeddings(train_df["text"].tolist())
y_train = train_df["label"].values

print("Extracting TEST embeddings...")
X_test = get_cls_embeddings(test_df["text"].tolist())
y_test = test_df["label"].values

print("Shapes:", X_train.shape, X_test.shape)

# ======================================================
# 5. TRAIN SVM CLASSIFIER
# ======================================================
clf = LinearSVC(C=1.0)
clf.fit(X_train, y_train)

# ======================================================
# 6. EVALUATE
# ======================================================
preds = clf.predict(X_test)

acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
cm = confusion_matrix(y_test, preds)

print("\n=== FINAL RESULTS ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\nConfusion Matrix:")
print(cm)

print("\nDetailed Report:")
print(classification_report(y_test, preds))


/tmp/ipykernel_20/1513292650.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('Trigger', group_keys=False).apply(


Train: 8000  Test: 2000


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

100%|██████████| 250/250 [00:13<00:00, 19.02it/s]


Extracting TEST embeddings...


100%|██████████| 63/63 [00:03<00:00, 18.37it/s]


Shapes: (8000, 768) (2000, 768)

=== FINAL RESULTS ===
Accuracy:  0.6760
Precision: 0.6667
Recall:    0.7040
F1 Score:  0.6848

Confusion Matrix:
[[648 352]
 [296 704]]

Detailed Report:
              precision    recall  f1-score   support

           0       0.69      0.65      0.67      1000
           1       0.67      0.70      0.68      1000

    accuracy                           0.68      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.68      0.68      0.68      2000



/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
